In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
color = sns.color_palette()
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()

from sklearn.model_selection import train_test_split 
import lightgbm as lgb
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  

import cufflinks as cf
cf.go_offline()

import xgboost as xgb 


In [ ]:
#Load the tran and test dataframes 
application_train=
application_test=

In [ ]:
#Features dataframe using train and test set 
def featuresDF(df_train,df_test,target):
    training=df_train
    testing=df_test

    training['is_train']=1
    training['is_test']=0
    testing['is_train']=0
    testing['is_test']=1
    
    train_X=training.drop([target],axis=1)
    test_X=testing
    data=pd.concat([train_X,test_X],axis=0)
    return data
data=featuresDF(application_train,application_test,"Comment")

In [ ]:
#Target variable extraction 
def target(df_train,target):
    training=df_train
    Y=training[target]
    return Y
    
Y=target(application_train,'TARGET')

In [ ]:
#Missing values Analysis 
def missingValuesAnalysis(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
df=missingValuesAnalysis(application_train)

In [ ]:
#Function to capture numerical variables 
def _get_numerical_features(df):
    feats=[col for col in list(df.columns) if df[col].dtype!='object']
    return feats 
numFeatures=_get_numerical_features(data)

In [ ]:
#Function to capture categorical variables 
def _get_categorical_features(df):
    feats=[col for col in list(df.columns) if df[col].dtype=='object']
    return feats 
cats=_get_categorical_features(data)

In [ ]:
#Function to factorize categorical variables 
def _factorize_categoricals(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df 
#data=_factorize_categoricals(data,cats)

In [ ]:
#Function to convert categorical variables into dummy variables 
def _get_dummies(df,cats):
    for col in cats:
         df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    return df
#data=_get_dummies(data,cats)

In [ ]:
#Doing factorize or dummy variable encoding 
def encoding(df,flag):
    data_cats = _get_categorical_features(df)
    if(flag==1):
        df = _factorize_categoricals(df,data_cats)
    else:
        df=_get_dummies(df,data_cats)
        df=df.drop(data_cats,axis=1)
    return df
data=encoding(data,0)

In [ ]:
ignoreFeatures = ['CUST_ID']
importantFeatures = [col for col in data.columns if col not in ignoreFeatures]

In [ ]:
#Splitting into train and test set 
ignoreFeatures = ['CUST_ID', 'is_train', 'is_test']
importantFeatures = [col for col in data.columns if col not in ignoreFeatures]
trainX = data[data['is_train'] == 1][importantFeatures]
testX = data[data['is_test'] == 1][importantFeatures]
x_train, x_val, y_train, y_val = train_test_split(trainX, Y, test_size=0.2, random_state=18)

# Methos 1 with Linerar kernal

In [ ]:
svclassifier = SVC(kernel='linear')  
svclassifier.fit(x_train, y_train)  

In [ ]:
y_pred = svclassifier.predict(x_val)  

In [ ]:
print(confusion_matrix(y_val,y_pred))  
print(classification_report(y_val,y_pred))  

In [ ]:
preds=svclassifier.predict(testX)  
test_id=application_test['CUST_ID']
sub_svm = pd.DataFrame()
sub_svm['CUST_ID'] = test_id
sub_svm['COMMENT'] = preds
sub_svm.to_csv("Prediction.csv", index=False)
sub_svm.head()

# Methos 2 with Ploynomial kernal

In [ ]:
svclassifier = SVC(kernel='poly', degree=8)  
svclassifier.fit(x_train, y_train)  

In [ ]:
y_pred = svclassifier.predict(x_val)  

In [ ]:
print(confusion_matrix(y_val, y_pred))  
print(classification_report(y_val, y_pred))  

In [ ]:
preds=svclassifier.predict(testX)  
test_id=application_test['CUST_ID']
sub_svm = pd.DataFrame()
sub_svm['CUST_ID'] = test_id
sub_svm['COMMENT'] = preds
sub_svm.to_csv("Prediction.csv", index=False)
sub_svm.head()

# Try sigmoid and gaussian kernal 